In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import numpy as np

In [ ]:
global PriceDataCache

def predictPrice:
    data = yf.download(tickers = 'TSLA', start = '2019-07-19',end = '2024-07-21')
    offset = pd.Timedelta(days=-30)
    
    # Add indicators...
    data['RSI']=ta.rsi(data.Close, length=3)
    # data['EMAF']=ta.ema(data.Close, length=3)
    # data['EMAM']=ta.ema(data.Close, length=6)
    # data['EMAS']=ta.ema(data.Close, length=9)
    data['DPO']=ta.dpo(data.Close, length=3, centered=True)
    
    # Add A Target Column
    data['Target'] = data['Adj Close']-data.Open
    data['Target'] = data['Target'].shift(-1)
    # Add a TargetClass Column...
    data['TargetClass'] = [1 if data['Target'][i]>0 else 0 for i in range(len(data))]
    # Add tje TargetMextC;pse Column...
    data['TargetNextClose'] = data['Adj Close'].shift(-1)
    
    # Remove Rows with Missing Data
    data.dropna(inplace=True)
    # Reindex the dataframe
    data.reset_index(inplace = True)
    
    # Drop Fields not needed for training input...
    # data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)
    data.drop(['Date'], axis=1, inplace=True)
    
    # Copy the data DF into a new data_set DF
    data_set = data.iloc[:, 0:11]#.values

    #Target column Categories
    # y: Red Candles
    y =[1 if data.Open[i]>data.Close[i] else 0 for i in range(0, len(data))]
    # y1: Green Candles
    yi = [data.Open[i]-data.Close[i] for i in range(0, len(data))]

    # Normalize the data_set DF from money values to a value between 0 and 1...
    sc = MinMaxScaler(feature_range=(0,1))
    data_set_scaled = sc.fit_transform(data_set)
    
    X = []

    backcandles = 30
    for j in range(8): # data_set_scaled[0].size):#2 columns are target not X
        X.append([])
        for i in range(backcandles, data_set_scaled.shape[0]): # backcandles+2
            X[j].append(data_set_scaled[i-backcandles:i, j])
    
    # Need to understand this area better...
    # move axis from 0 to position 2
    X=np.moveaxis(X, [0], [2])
    
    #Erase first elements of y because of backcandles to match X length
    #del(yi[0:backcandles])
    #X, yi = np.array(X), np.array(yi)
    # Choose -1 for last column, classification else -2...
    X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
    y=np.reshape(yi,(len(yi),1))
    #y=sc.fit_transform(yi)
    #X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    splitlimit = int(len(X)*0.8)
    X_train, X_test = X[:splitlimit], X[splitlimit:]
    y_train, y_test = y[:splitlimit], y[splitlimit:]
    np.random.seed(10)
    
    lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
    inputs = LSTM(150, name='first_layer')(lstm_input)
    inputs = Dense(1, name='dense_layer')(inputs)
    output = Activation('linear', name='output')(inputs)
    model = Model(inputs=lstm_input, outputs=output)
    adam = optimizers.Adam()
    model.compile(optimizer=adam, loss='mse')
    model.fit(x=X_train, y=y_train, batch_size=15, epochs=15, shuffle=True, validation_split = 0.1)
